# Sensivivity test analysis


In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from typing import Dict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yaml
import re
import xarray as xr
import matplotlib as mpl

import matplotlib.dates as mdates


from functools import reduce
from src.config import get_config, get_dask_cluster
from src.plotting import SensitivityGridPlot, SensitivityTestSummaryFigure
from src.analysis import (
    time_filter,
    time_to_datetimeindex,
    map_datasets_to_experiments,
    compute_experiment_modifications,
    aggregate_friction_velocities,
    compute_relative_humidities,
    compute_shf_to_net_rad,
    compute_response_factors,
    aggregate_ta_2m,
    compute_abs_diff_to_baseline,
    bbox_filter,
)
from src.job_generation import read_namelist

from functools import partial


config = get_config()
cluster, client = get_dask_cluster(config)

## Analysis preparation


### Define target spatial averaging area


In [ ]:
urban_offset = (2110 + 2 * 896 - 1536, 1280)

total_target_area_bbox = (
    urban_offset[0] + 256,
    urban_offset[1] + 256,
    urban_offset[0] + 1536 - 256,
    urban_offset[1] + 1536 - 256,
)

In [ ]:
total_target_area_filter = partial(bbox_filter, bbox=total_target_area_bbox)

In [ ]:
daytime_filter = partial(
    time_filter,
    start_time=np.datetime64("2018-03-30T12:00"),
    end_time=np.datetime64("2018-03-30T16:00"),
)
nighttime_filter = partial(
    time_filter,
    start_time=np.datetime64("2018-03-31T00:00"),
    end_time=np.datetime64("2018-03-31T04:00"),
)

### Load baseline


In [ ]:
baseline_p3d = read_namelist(Path(config.path.experiments.sensitivity) / "base_p3d.yml")
origin_date_time = pd.Timestamp(
    baseline_p3d["initialization_parameters"]["origin_date_time"]
)
output_path = Path(config.path.data.jobs) / "slurb_s_base" / "OUTPUT"
baseline_outputs = {
    "av_3d": xr.open_dataset(
        output_path / "slurb_s_base_av_3d.000.nc", chunks={"time": "auto"}
    ),
    "av_xy": xr.open_dataset(
        output_path / "slurb_s_base_av_xy.000.nc", chunks={"time": "auto"}
    ),
    "av_xz": xr.open_dataset(
        output_path / "slurb_s_base_av_xz.000.nc", chunks={"time": "auto"}
    ),
    "pr": xr.open_dataset(
        output_path / "slurb_s_base_pr.000.nc", chunks={"time": "auto"}
    ),
    "ts": xr.open_dataset(
        output_path / "slurb_s_base_ts.000.nc", chunks={"time": "auto"}
    ),
    "xy": xr.open_dataset(
        output_path / "slurb_s_base_xy.000.nc", chunks={"time": "auto"}
    ),
}
for dataset_name in baseline_outputs.keys():
    baseline_outputs[dataset_name] = time_to_datetimeindex(
        baseline_outputs[dataset_name], origin_date_time
    )
    # Offset aggregation period labels to period center
    if len(dataset_name.split("_")) > 1:
        baseline_outputs[dataset_name]["time"] = baseline_outputs[dataset_name][
            "time"
        ] - pd.Timedelta(15, "m")
    baseline_outputs[dataset_name]["second_of_day"] = (
        baseline_outputs[dataset_name].time.dt.hour * 3600
        + baseline_outputs[dataset_name].time.dt.minute * 60
        + baseline_outputs[dataset_name].time.dt.second
    )
slurb_driver_path = Path(config.path.data.jobs) / "slurb_s_base" / "INPUT"
baseline_slurb_driver = xr.open_dataset(
    slurb_driver_path / "slurb_s_base_slurb",
)

### Load experiments


In [ ]:
with open(Path(config.path.experiments.sensitivity) / "experiments.yml", "r") as cfile:
    experiments = yaml.safe_load(cfile)

Experiment definitions don't contain information from job names (positive and negative modification). These are added to the definitions here.


In [ ]:
jobs = [job.name for job in Path(config.path.data.jobs).iterdir()]

In [ ]:
for experiment_name in experiments.keys():
    pattern_pos = re.compile(f"slurb_s_{experiment_name}\\+.*")
    pattern_neg = re.compile(f"slurb_s_{experiment_name}\\-.*")
    for job in jobs:
        if pattern_pos.search(job):
            experiments[experiment_name]["job_name_positive"] = job
        elif pattern_neg.search(job):
            experiments[experiment_name]["job_name_negative"] = job

Set dataset objects for the outputs. To assist bulk computations, store these in flattened dictionary as well. These are lazy-loaded so no worries with memory consumption.


In [ ]:
datasets_all = map_datasets_to_experiments(experiments, origin_date_time)

Compute the absolute and relative modifications done for each experiment. First, load the baseline values. The baseline values for radiation and wind speed measurements need to be computed from the dynamic driver. We use the daily median value as a reference value.


In [ ]:
with open(
    Path(config.path.experiments.sensitivity) / "base_slurb_driver.yml", "r"
) as cfile:
    baseline_values = yaml.safe_load(cfile)

In [ ]:
baseline_dynamic_path = (
    Path(config.path.data.jobs) / "slurb_s_base" / "INPUT" / "slurb_s_base_dynamic"
)
baseline_dynamic = xr.open_dataset(baseline_dynamic_path)
baseline_values["wspeed"] = {}
baseline_values["rad_sw_in"] = {}
baseline_values["rad_lw_in"] = {}
baseline_values["wspeed"]["value"] = float(
    np.sqrt(
        baseline_dynamic["init_atmosphere_u"].isel(z=-1).mean() ** 2
        + baseline_dynamic["init_atmosphere_v"].isel(z=-1).mean() ** 2
    )
)
baseline_values["rad_sw_in"]["value"] = float(baseline_dynamic["rad_sw_in"].mean())
baseline_values["rad_lw_in"]["value"] = float(baseline_dynamic["rad_lw_in"].mean())

In [ ]:
baseline_slurb_path = (
    Path(config.path.data.jobs) / "slurb_s_base" / "INPUT" / "slurb_s_base_slurb"
)
baseline_slurb = xr.open_dataset(baseline_slurb_path)
baseline_values["deep_soil_temperature"] = {}
baseline_values["deep_soil_temperature"]["value"] = float(
    baseline_slurb["deep_soil_temperature"].mean()
)

In [ ]:
compute_experiment_modifications(experiments, baseline_values)

In [ ]:
baseline_values["deep_soil_temperature"]["value"]

### Target variable definitions


In [ ]:
targets_daytime = {
    r"shf_day": {
        "symbol": r"H",
        "group": "Daytime",
        "units": r"${}_{\left(\mathrm{W}~\mathrm{m}^{-2}\right)}$",
    },
    r"qsws_day": {
        "symbol": r"LE",
        "group": "Daytime",
        "units": r"${}_{\left(\mathrm{W}~\mathrm{m}^{-2}\right)}$",
    },
    r"ta_2m_day": {
        "symbol": r"T_{\mathrm{2m}}",
        "group": "Daytime",
        "units": r"${}_{\left(\mathrm{K}\right)}$",
    },
    r"slurb_t_c_day": {
        "symbol": r"T_{C}",
        "group": "Daytime",
        "units": r"${}_{\left(\mathrm{K}\right)}$",
    },
    r"slurb_rh_can_day": {
        "symbol": r"RH_{\mathrm{can}}",
        "group": "Daytime",
        "units": r"${}_{\left(\mathrm{p.p}\right)}$",
    },
    r"us_day": {
        "symbol": r"u_*",
        "group": "Daytime",
        "units": r"${}_{\left(\mathrm{m}~\mathrm{s}^{-1}\right)}$",
    },
}
targets_nighttime = {
    r"shf_night": {
        "symbol": r"H",
        "group": "Nighttime",
        "units": r"${}_{\left(\mathrm{W}~\mathrm{m}^{-2}\right)}$",
    },
    r"qsws_night": {
        "symbol": r"LE",
        "group": "Nighttime",
        "units": r"${}_{\left(\mathrm{W}~\mathrm{m}^{-2}\right)}$",
    },
    r"ta_2m_night": {
        "symbol": r"T_{\mathrm{2m}}",
        "group": "Nighttime",
        "units": r"${}_{\left(\mathrm{K}\right)}$",
    },
    r"slurb_t_c_night": {
        "symbol": r"T_{C}",
        "group": "Nighttime",
        "units": r"${}_{\left(\mathrm{K}\right)}$",
    },
    r"slurb_rh_can_night": {
        "symbol": r"RH_{\mathrm{can}}",
        "group": "Nighttime",
        "units": r"${}_{\left(\mathrm{p.p}\right)}$",
    },
    r"us_night": {
        "symbol": r"u_*",
        "group": "Nighttime",
        "units": r"${}_{\left(\mathrm{m}~\mathrm{s}^{-1}\right)}$",
    },
}
targets_diurnal = {
    r"hysteresis_index_diurnal": {
        "symbol": r"HI",
        "group": "Diurnal",
        "units": r"${}_{\left(\mathrm{m}~\mathrm{s}^{-1}\right)}$",
    },
}
targets = {**targets_daytime, **targets_nighttime}  # **targets_diurnal

### Compute further diagnostic variables


Aggregate urban and non-urban friction velocities (not done by PALM due to technical reasons).


In [ ]:
aggregate_friction_velocities(experiments, baseline_outputs, baseline_slurb_driver)
aggregate_ta_2m(experiments, baseline_outputs, baseline_slurb_driver)
compute_relative_humidities(experiments, baseline_outputs)
compute_shf_to_net_rad(experiments, baseline_outputs)

## Results


### Compute response factors

These are done for so called parameter experiements


In [ ]:
parameter_experiments = dict(
    filter(
        lambda item: item[1].get("subcategory", "")
        in ["material_parameters", "urban_morphology"],
        experiments.items(),
    )
)
# targets = targets_daytime
cols = list(targets.keys())
long_names = [exp["long_name"] for exp in parameter_experiments.values()]
index = pd.Index(long_names, name="Parameter")
cells = np.full((len(parameter_experiments), len(targets)), 0.0)

In [ ]:
parameter_data = pd.DataFrame(cells, index=index, columns=cols)
parameter_data.insert(0, "ID", parameter_experiments.keys())

In [ ]:
parameter_data = parameter_data.apply(
    compute_response_factors,
    axis=1,
    experiments=experiments,
    spatial_filtering=total_target_area_filter,
    daytime_filter=daytime_filter,
    nighttime_filter=nighttime_filter,
)

Compute factors for forcing experiments.


In [ ]:
forcing_experiments = dict(
    filter(
        lambda item: item[1].get("subcategory", "")
        in ["external_forcing", "radiation"],
        experiments.items(),
    )
)
long_names = [exp["long_name"] for exp in forcing_experiments.values()]
index = pd.Index(long_names, name="Forcing")
cells = np.full((len(forcing_experiments), len(targets)), 0.0)

forcing_data = pd.DataFrame(cells, index=index, columns=cols)
forcing_data.insert(0, "ID", forcing_experiments.keys())
forcing_data = forcing_data.apply(
    compute_response_factors,
    axis=1,
    experiments=experiments,
    spatial_filtering=total_target_area_filter,
    daytime_filter=daytime_filter,
    nighttime_filter=nighttime_filter,
)

### Compute simple difference for parametrisations


In [ ]:
parametrisation_experiments = dict(
    filter(
        lambda item: item[1].get("category", "") in ["namelist"],
        experiments.items(),
    )
)
long_names = [exp["long_name"] for exp in parametrisation_experiments.values()]
index = pd.Index(long_names, name="Parametrisation")
cells = np.full((len(parametrisation_experiments), len(targets)), 0.0)

parametrisation_data = pd.DataFrame(cells, index=index, columns=cols)
parametrisation_data.insert(0, "ID", parametrisation_experiments.keys())
parametrisation_data = parametrisation_data.apply(
    compute_abs_diff_to_baseline,
    axis=1,
    baseline_outputs=baseline_outputs,
    experiments=experiments,
    spatial_filtering=total_target_area_filter,
    daytime_filter=daytime_filter,
    nighttime_filter=nighttime_filter,
)

In [ ]:
parametrisation_data

In [ ]:
parameter_data

In [ ]:
forcing_data

Compute max response in orsder to harmonize colorscale.


In [ ]:
max_response_factor = (
    pd.concat([parameter_data, forcing_data, parametrisation_data])
    .drop("ID", axis=1)
    .abs()
    .max(axis=0)
)

### Plot results


In [ ]:
gridplot = SensitivityGridPlot(
    parameter_data.drop("ID", axis=1), targets, range=max_response_factor
)
gridplot.plot()

In [ ]:
gridplot.fig.savefig("results/sensitivity_parameters.pdf", dpi=300)

In [ ]:
gridplot = SensitivityGridPlot(
    forcing_data.drop("ID", axis=1), targets, range=max_response_factor
)
gridplot.plot()

In [ ]:
gridplot.fig.savefig("results/sensitivity_forcing.pdf")

In [ ]:
gridplot = SensitivityGridPlot(
    parametrisation_data.drop("ID", axis=1), targets, range=max_response_factor
)
gridplot.plot()

In [ ]:
gridplot.fig.savefig("results/sensitivity_parametrisations.pdf")

### Example diurnal cycle

Plot an example of a diurnal cycle for urban fraction experiments. Rest will will go to supplementary.


In [ ]:
summary = SensitivityTestSummaryFigure(
    experiments["fr_urb"], baseline_outputs, total_target_area_filter
)
summary.plot()
custom_lines = [
    plt.Line2D([0], [0], color="tab:red"),
    plt.Line2D([0], [0], color="tab:grey"),
    plt.Line2D([0], [0], color="tab:blue"),
]
summary.fig.suptitle("")
summary.fig.legend(
    custom_lines,
    [
        r"$\mathcal{A}_{\mathrm{urb}}=0.9$",
        r"$\mathcal{A}_{\mathrm{urb}}=0.8$",
        r"$\mathcal{A}_{\mathrm{urb}}=0.7$",
    ],
    loc="upper center",
    ncol=3,
    bbox_to_anchor=(0.55, 1.05),
)
summary.fig.savefig(
    "results/sensitivity_urban_fraction_dirunal.pdf",
    pad_inches=0.0,
    bbox_inches="tight",
)